In [1]:
import cv2
import numpy as np
from math import ceil,floor

In [2]:
vidObj = cv2.VideoCapture('movie.mov') 

frames = []
input_frames = []

count = 0
success = 1

while success: 
    success, image = vidObj.read()     
    if image is None:
        break
    
    
    frames.append(image)
    input_frames.append(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)/255)

In [3]:
reference = cv2.imread('reference.jpg', cv2.IMREAD_GRAYSCALE)/255

In [4]:
def ExhaustiveSearch(t,r):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    
    for m in range(I-M+1): #
        for n in range(J-N+1): #
            current_correlation = 0
            for i in range(m,m+M-1):
                for j in range(n,n+N-1):
                    current_correlation += t[i][j]*r[i-m][j-n]
                    
            if(m%100==0):
                if(n%100==0):
                    print(m,n,current_correlation)
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
    
    return best_node,max_correlation

In [5]:
#[X,Y],max_correlation = ExhaustiveSearch(input_frames[0],reference)
[X,Y],max_correlation = [247,137], 3588.946220684356
M,N = reference.shape[0],reference.shape[1]

print(X,Y,max_correlation)

247 137 3588.946220684356


In [6]:
def ExhaustiveSearchUsingCenter(t,r,X,Y,p,previous_correlation):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [0,0]
    max_correlation = 10e10
    
    for m in range(X-p,X+p): #
        for n in range(Y-p,Y+p): #
            current_correlation = 0
            for i in range(m,m+M-1):
                for j in range(n,n+N-1):
                    current_correlation += t[i][j]*r[i-m][j-n]
                    
            if(previous_correlation>=current_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
                break
            
            if(current_correlation<max_correlation):
                max_correlation = current_correlation
                best_node = [m,n]
    
    return best_node,max_correlation

In [7]:
def twoDimensionalLogarithmicSearch(t,r,X,Y,p):
    M,N = r.shape[0],r.shape[1]
    I,J = t.shape[0],t.shape[1]
    
    count = 0
    
    best_node = [X,Y]
    
    max_correlation = 10e10
    
    div = p
    
    while True:
        arr = [floor(-div/2),0,ceil(div/2)]
                
        #print(max_correlation)
        
        for m in arr:
            for n in arr:                
                x,y = X+m,Y+n
                
                if(x==X and y==Y):
                    continue
                
                current_correlation = 0
                
                for i in range(x,x+M-1):
                    for j in range(y,y+N-1):
                        current_correlation += t[i][j]*r[i-x][j-y]
                
                if(current_correlation<max_correlation):
                    max_correlation = current_correlation
                    best_node = [x,y]
                
        [X,Y] = best_node

        div = int(div/2)
        
        if(div<1):
            break
    
    return best_node,max_correlation


In [8]:
def outline_creator(frame,center_x,center_y,M,N):
    for i in range(center_x-5,center_x+M+5):
        frame[i][center_y] = [0,0,255]
        frame[i][center_y+N+5] = [0,0,255]
    
    for j in range(center_y,center_y+N+5):
        frame[center_x-5][j] = [0,0,255]
        frame[center_x+M+5][j] = [0,0,255]
            
    return frame    

In [9]:
def output_frames(frames,input_frames,reference,X,Y,p,first_frame_output,max_correlation):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation = ExhaustiveSearchUsingCenter(input_frames[i],reference,X,Y,p,max_correlation)
        print([X,Y],max_correlation)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output

In [10]:
#twoDimensionalLogarithmicSearch(input_frames[1],reference,247,137,7)

def output_frames_2DLogSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation = twoDimensionalLogarithmicSearch(input_frames[i],reference,X,Y,p)
        print([X,Y],max_correlation)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output

In [11]:
def filter_scale(img,scale,filter=False):
    scale_percent = 100/scale # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    if filter:
        img = cv2.GaussianBlur(img,(5,5),0)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        #converted = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
        return resized
    else:
        return img

In [12]:
def hierarchicalSearch(t,r,X,Y,p):
    
    frames = []
    refs = []
    
    for i in range(3):
        scale = pow(2,i)
        
        if(scale>1):
            frames.append(filter_scale(t,scale,filter=True))
            refs.append(filter_scale(r,scale,filter=True))
        else:
            frames.append(t)
            refs.append(r)

    # Level 2
    
    [x1,y1],max_correlation = twoDimensionalLogarithmicSearch(frames[2],refs[2],X//4,Y//4,p//4)
    
    [x1,y1] = [x1-X//4,y1-Y//4]
    
    # Level 1
    
    [x2,y2],max_correlation = twoDimensionalLogarithmicSearch(frames[1],refs[1],(X//2)+2*x1,(Y//2)+2*y1,1)
    
    [x2,y2] = [x2-(X//2),y2-(Y//2)]

    # Level 0
    
    best_node,max_correlation = twoDimensionalLogarithmicSearch(frames[0],refs[0],(X)+2*x2,(Y)+2*y2,1)
    
    return best_node,max_correlation

def output_frames_hierarchicalSearch(frames,input_frames,reference,X,Y,p,first_frame_output):
    output = []
    output.append(first_frame_output)
    M,N = reference.shape[0],reference.shape[1]
    
    for i in range(1,len(frames)):
        [X,Y],max_correlation = hierarchicalSearch(input_frames[i],reference,X,Y,p)
        print([X,Y],max_correlation)
        output.append(outline_creator(frames[i],X,Y,M,N))
    
    return output

In [13]:
first_frame_output = outline_creator(frames[0],X,Y,M,N)

# EXHAUSTIVE

#output_frames = output_frames(frames,input_frames,reference,X,Y,2,first_frame_output,max_correlation)

# 2D LOGARITHMIC

#output_frames = output_frames_2DLogSearch(frames,input_frames,reference,X,Y,2,first_frame_output)

output_frames = output_frames_hierarchicalSearch(frames,input_frames,reference,X,Y,2,first_frame_output)

[246, 138] 3597.3671818531466
[247, 139] 3586.9623529411797
[246, 140] 3600.4933179546397
[247, 141] 3589.078216070753
[246, 142] 3600.38346789697
[247, 143] 3589.657623990784
[246, 144] 3599.8263898500727
[247, 145] 3588.946220684356
[246, 146] 3597.3671818531466
[247, 147] 3586.9623529411797
[246, 148] 3600.4933179546397
[247, 149] 3589.078216070753
[246, 150] 3600.38346789697
[247, 151] 3589.657623990784
[246, 152] 3599.8263898500727
[247, 153] 3588.946220684356
[246, 154] 3597.3671818531466
[247, 155] 3586.9623529411797
[246, 156] 3600.4933179546397
[247, 157] 3589.078216070753
[246, 158] 3600.38346789697
[247, 159] 3589.657623990784
[246, 160] 3599.8263898500727
[247, 161] 3588.946220684356
[246, 162] 3597.3671818531466
[247, 163] 3586.9623529411797
[246, 164] 3600.4933179546397
[247, 165] 3589.078216070753
[246, 166] 3600.38346789697
[247, 167] 3589.657623990784
[246, 168] 3599.8263898500727
[247, 169] 3588.946220684356
[246, 170] 3597.3671818531466
[247, 171] 3586.9623529411797


[154, 326] 3588.4271434063826
[153, 325] 3604.263514033065
[152, 326] 3589.3857285659356
[151, 325] 3605.0163168012355
[150, 326] 3590.2424452133832
[149, 325] 3604.7853748558296
[148, 326] 3589.2700346020806
[147, 325] 3602.271664744329
[146, 326] 3588.4271434063826
[145, 325] 3604.263514033065
[144, 326] 3589.3857285659356
[143, 325] 3605.0163168012355
[142, 326] 3590.2424452133832
[141, 325] 3604.7853748558296
[140, 326] 3589.2700346020806
[139, 325] 3602.271664744329
[138, 326] 3588.4271434063826
[137, 325] 3604.263514033065
[136, 326] 3589.3857285659356
[135, 325] 3605.0163168012355
[134, 326] 3590.2424452133832
[133, 325] 3604.7853748558296
[132, 326] 3589.2700346020806
[131, 325] 3602.271664744329
[130, 326] 3588.4271434063826
[129, 325] 3604.263514033065
[128, 326] 3589.3857285659356
[127, 325] 3605.0163168012355
[126, 326] 3590.2424452133832
[125, 325] 3604.7853748558296
[124, 326] 3589.2700346020806
[123, 325] 3602.271664744329
[122, 326] 3588.4271434063826
[121, 325] 3604.26

[58, 139] 3588.8884890426866
[58, 138] 3589.294179161876
[58, 137] 3587.5606459054184
[59, 137] 3589.5474356016975
[60, 137] 3590.2430449827075
[61, 137] 3591.1151249519403
[62, 137] 3592.390034602077
[63, 137] 3591.7645982314557
[64, 137] 3591.0582545175
[65, 137] 3590.061084198391
[66, 137] 3588.809211841605
[67, 137] 3589.5474356016975
[68, 137] 3590.2430449827075
[69, 137] 3591.1151249519403
[70, 137] 3592.390034602077
[71, 137] 3591.7645982314557
[72, 137] 3591.0582545175
[73, 137] 3590.061084198391
[74, 137] 3588.809211841605
[75, 137] 3589.5474356016975
[76, 137] 3590.2430449827075
[77, 137] 3591.1151249519403
[78, 137] 3592.390034602077
[79, 137] 3591.7645982314557
[80, 137] 3591.0582545175
[81, 137] 3590.061084198391
[82, 137] 3588.809211841605
[83, 137] 3589.5474356016975
[84, 137] 3590.2430449827075
[85, 137] 3591.1151249519403
[86, 137] 3592.390034602077
[87, 137] 3591.7645982314557
[88, 137] 3591.0582545175
[89, 137] 3590.061084198391
[90, 137] 3588.809211841605
[91, 137] 

In [14]:
cap = cv2.VideoCapture('movie.mov')
fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output.avi', fourcc, 760/13, (int(cap.get(3)),int(cap.get(4))))


for frame in output_frames:
    out.write(frame)
cap.release()
out.release()